### CMSC 170: Laboratory Exercise 2
##### Missionaries and Cannibals
Salcedo, Chris Samuel

2022-05055

#### Task 1.A: Define the Problem

##### 1. What is the problemt that needs a solution?

The 8-puzzle problem is a sliding puzzle that consists of 8-numbered tiles, labeled 1-8, and an empty space in a 3x3 grid. The challenge is rearranging the tiles through that empty space to achieve a specific goal state.

##### 2. What is the initial state?

The initial state is any valid state/configuration of the problem, serving as the starting point to where the problem-solving begins.

##### 3. What is the goal state?

The goal state is the desired state/configuration that is to be achieved. The most common goal state is the numerically arranged state [[1,2,3],[4,5,6],[7,8,0]].

##### 4. What are the valid actions?
The valid actions consist of four moves, representing the moves of the tiles into the empty space.

* UP: Move the tile below the empty space upward.
* DOWN: Move the tile above the empty space doenward
* LEFT: Move the tile on the right of the empty space to the left
* RIGHT: Move the tile on the left of the empty space to the right

Of course, the validity of a move or an action relies on the current condition of the empty space. If the empty space is located in a side or corner of the grid then there will not be enough tiles arround it to fullfill the allowability of the four listed moves.

##### 5. Whatis/are the functions that validate whether a state is valid?

State validation check include constraints where,
1. The board should only contain exactly 8 numbered tiles and one empty space
2. All tiles are within the 3x3 boundaries
3. No duplicate numbers exist, and
4. The state is reachable from the initial state

#### References:
* https://www.geeksforgeeks.org/dsa/8-puzzle-problem-using-branch-and-bound/

#### Task 1.B: Define the Breadth-First Search

##### 1. What is the main idea behind the breadth-first search (BFS) algorithm, and how does it explore a graph or tree?

The Breadth-First Search (BFS) is a search algorithm that explores nodes by level, starting from the root note, visiting all nodes at depth `d` before exploring nodes `d+1`. In the current puzzle, the algorithm explores all possible moves from the current state before moving to states that require more moves. This way, the shortest possible path is guaranteed to be outputted.

##### 2. What data structure does BFS use, and why is it important? How does BFS differ from depth-first search (DFS)?

BFS uses a queue (FIFO) data structure to ensure that nodes are processed in the order they were discovered and therefore guaranteeing level-by-level exploration. On the other hand, DFS uses stack (LIFO) and explores more depth-wise (hence the name), which although may find a solution that is not the most optimized, it is better memory-wise. BFS is used mostly on problems such as bipartite graphs and shortest paths, while DFS shows its strengths on problems such as acyclic graphs and finding strongly connected components. 

#### 3. What is the time complexity of BFS for a graph with V vertices and E edges?

The time complexity of BFS is O(V+E) while its space complexity is O(V) for storing the queue and visited states.

#### References:
* https://www.geeksforgeeks.org/dsa/breadth-first-search-or-bfs-for-a-graph/
* https://www.geeksforgeeks.org/dsa/difference-between-bfs-and-dfs/


#### Task 2: 8-Puzzle Game in Python

In [27]:
import numpy as np
from collections import deque
import copy

class EightPuzzleProblem:

    def __init__(self):
        self.board = None
        self.goal_state = [[1,2,3], [4,5,6], [7,8,0]]

        self.empty_pos = None

    def display_instructions(self):
        print()
        print("\nThe 8-puzzle problem is a 3x3 board with 8 tiles numbered from 1 to 8")
        print("and one empty space (represented by 0).")
        print("\nThe objective is to begin with an arbitrary configuration of tiles,")
        print("and move them to place the numbered tiles to match the final configuration.")
        print("\n📋 RULES:")
        print("1. Input the initial state of the puzzle using this format:")
        print("   ➢ [1,2,3,4,0,8,5,6,7] (where 0 represents the empty space)")
        print("\n2. Use the following keys to move tiles into the empty space:")
        print("   'W' or 'w' → Move tile UP into empty space")
        print("   'S' or 's' → Move tile DOWN into empty space") 
        print("   'A' or 'a' → Move tile LEFT into empty space")
        print("   'D' or 'd' → Move tile RIGHT into empty space")
        print("   'Q' or 'q' → Quit the game")
        print()

    def initial_state(self):
        while True:
            try:
                print("Enter initial state of the puzzle: ")
                user_input = input("Format: [1,2,3,4,0,8,5,6,7]: ")

                numbers = eval(user_input)
                if len(numbers) != 9:
                    raise ValueError("Please input 9 numbers (0-8).")

                self.board = [numbers[i:i+3] for i in range(0, 9, 3)]

                if self.validate_state(self.board):
                    self.find_empty()
                    self.display_board()
                    return True
                else:
                    print("Invalid state! Please ensure that the input aligns with the rules of the problem")
            except Exception as e:
                print(f"Invalid input format | Error: {e}")
                print("Please use the format: [1,2,3,4,0,8,5,6,7]")

    def validate_state(self, board):
        flat = [num for row in board for num in row]
        return sorted(flat) == list(range(9))

    def find_empty(self):
        # position of empty space
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    self.empty_pos = (i, j)
                    return

    def display_board(self):
        print()
        print("┌─────┬─────┬─────┐")
        for i, row in enumerate(self.board):
            row_str = "│"
            for num in row:
                if num == 0:
                    row_str += "     │"
                else:
                    row_str +=  f"  {num}  │"
            print(row_str)
            if i < 2:
                print("├─────┼─────┼─────┤")
        print("└─────┴─────┴─────┘")

    def get_valid(self):
        row, col = self.empty_pos
        valid_moves = []

        directions = {
            'UP': (1, 0, 'W'),
            'DOWN': (-1, 0, 'S'),
            'LEFT': (0, 1, 'A'),
            'RIGHT': (0, -1, 'D')
        }

        for direction, (dr, dc, key) in directions.items():
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                valid_moves.append((direction, key, new_row, new_col))

        return valid_moves

    def make_move(self, direction):
        valid_moves = self.get_valid()
        move_map = {move[1].lower(): move for move in valid_moves}

        if direction.lower() in move_map:
            _, _, tile_row, tile_col = move_map[direction.lower()]
            empty_row, empty_col = self.empty_pos

            self.board[empty_row][empty_col] = self.board[tile_row][tile_col]
            self.board[tile_row][tile_col] = 0
            self.empty_pos = (tile_row, tile_col)

            return True
        return False

    def is_goal(self):
        return self.board == self.goal_state

    def play(self):

        self.display_instructions()

        if not self.initial_state():
            return

        moves_count = 0

        while not self.is_goal():
            move = input("\nEnter your move: ").strip()
            
            if move.lower() == 'q':
                print("Thanks for playing!")
                break

            if self.make_move(move):
                moves_count += 1
                print(f"\nMove {moves_count}")

                self.display_board()

                if self.is_goal():
                    print("CONGRATULATIONS!")
                    print(f"You solved the puzzle in {moves_count} moves!")
                    break
            else:
                print("Invalid move, try again.")

In [28]:
game = EightPuzzleProblem()
game.play()



The 8-puzzle problem is a 3x3 board with 8 tiles numbered from 1 to 8
and one empty space (represented by 0).

The objective is to begin with an arbitrary configuration of tiles,
and move them to place the numbered tiles to match the final configuration.

📋 RULES:
1. Input the initial state of the puzzle using this format:
   ➢ [1,2,3,4,0,8,5,6,7] (where 0 represents the empty space)

2. Use the following keys to move tiles into the empty space:
   'W' or 'w' → Move tile UP into empty space
   'S' or 's' → Move tile DOWN into empty space
   'A' or 'a' → Move tile LEFT into empty space
   'D' or 'd' → Move tile RIGHT into empty space
   'Q' or 'q' → Quit the game

Enter initial state of the puzzle: 


Format: [1,2,3,4,0,8,5,6,7]:  [1,2,3,4,0,8,5,6,7]



┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │     │  8  │
├─────┼─────┼─────┤
│  5  │  6  │  7  │
└─────┴─────┴─────┘



Enter your move:  w



Move 1

┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │  6  │  8  │
├─────┼─────┼─────┤
│  5  │     │  7  │
└─────┴─────┴─────┘


...


Move 161

┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │  6  │     │
├─────┼─────┼─────┤
│  7  │  8  │  5  │
└─────┴─────┴─────┘



Enter your move:  d



Move 162

┌─────┬─────┬─────┐
│  1  │  2  │  3  │
├─────┼─────┼─────┤
│  4  │     │  6  │
├─────┼─────┼─────┤
│  7  │  8  │  5  │
└─────┴─────┴─────┘



Enter your move:  q


Thanks for playing!
